In [2]:
import pandas as pd
import numpy as np
from sodapy import Socrata

#settings
pd.set_option('display.max_columns', 500)


In [4]:
client = Socrata("chronicdata.cdc.gov", 'phceef5Bs3FyOOOd82FyjwziG', username='mfarme@outlook.com', password='u7hwEt8ChspWWh')
results = client.get_all("swc5-untb") # PLACES 2023 Model-based county estimates (using BRFSS 2021,2020 data)
results = pd.DataFrame.from_records(results)
results.drop([
    'datasource', 'data_value_unit', 'data_value_type', 'locationid',
    'categoryid', 'geolocation', ':@computed_region_skr5_azej',
    'short_question_text','category'
], axis=1, inplace=True) 

In [34]:
df = results.copy() 
df = df[df['year'] == '2021'] #2021 BRFSS Data Only
#Choice of variables for analysis 
df = df[df['measureid'].isin(['DIABETES', 'GHLTH','CSMOKING', 'OBESITY', 'CHECKUP', 'DEPRESSION', 'ACCESS2', 'MHLTH'])] 
df['m'] = df['measureid'].astype(str) + '_' + df['datavaluetypeid'].astype(str) 
df.drop(['measureid', 'datavaluetypeid'], axis=1, inplace=True)

# Create columns for each measure
df_transformed = pd.get_dummies(df, columns=['m'])

In [35]:
measures = [
    'm_ACCESS2_AgeAdjPrv', 'm_ACCESS2_CrdPrv',
    'm_CHECKUP_AgeAdjPrv', 'm_CHECKUP_CrdPrv', 'm_CSMOKING_AgeAdjPrv',
    'm_CSMOKING_CrdPrv', 'm_DEPRESSION_AgeAdjPrv', 'm_DEPRESSION_CrdPrv',
    'm_DIABETES_AgeAdjPrv', 'm_DIABETES_CrdPrv', 'm_GHLTH_AgeAdjPrv',
    'm_GHLTH_CrdPrv', 'm_MHLTH_AgeAdjPrv', 'm_MHLTH_CrdPrv',
    'm_OBESITY_AgeAdjPrv', 'm_OBESITY_CrdPrv'
]

In [36]:
for i in measures:
    # Set data value for each measure
    df_transformed[i] = np.where(df_transformed[i] == 1, df_transformed['data_value'], np.nan)
    # Set low and high confidence limit for each measure when data value is not null
    df_transformed[i + '_low'] = np.where(df_transformed[i] == df_transformed['data_value'], df_transformed['low_confidence_limit'], np.nan)
    df_transformed[i + '_high'] = np.where(df_transformed[i] == df_transformed['data_value'], df_transformed['high_confidence_limit'], np.nan)

df_transformed = df_transformed.drop(columns=['data_value', 'low_confidence_limit', 'high_confidence_limit', 'measure', 'year'])

In [37]:
df_group = df_transformed.groupby([ 'stateabbr', 'statedesc', 'locationname', 'totalpopulation']).sum().reset_index()
df_group.head(10)

,stateabbr,statedesc,locationname,totalpopulation,m_ACCESS2_AgeAdjPrv,m_ACCESS2_CrdPrv,m_CHECKUP_AgeAdjPrv,m_CHECKUP_CrdPrv,m_CSMOKING_AgeAdjPrv,m_CSMOKING_CrdPrv,m_DEPRESSION_AgeAdjPrv,m_DEPRESSION_CrdPrv,m_DIABETES_AgeAdjPrv,m_DIABETES_CrdPrv,m_GHLTH_AgeAdjPrv,m_GHLTH_CrdPrv,m_MHLTH_AgeAdjPrv,m_MHLTH_CrdPrv,m_OBESITY_AgeAdjPrv,m_OBESITY_CrdPrv,m_ACCESS2_AgeAdjPrv_low,m_ACCESS2_AgeAdjPrv_high,m_ACCESS2_CrdPrv_low,m_ACCESS2_CrdPrv_high,m_CHECKUP_AgeAdjPrv_low,m_CHECKUP_AgeAdjPrv_high,m_CHECKUP_CrdPrv_low,m_CHECKUP_CrdPrv_high,m_CSMOKING_AgeAdjPrv_low,m_CSMOKING_AgeAdjPrv_high,m_CSMOKING_CrdPrv_low,m_CSMOKING_CrdPrv_high,m_DEPRESSION_AgeAdjPrv_low,m_DEPRESSION_AgeAdjPrv_high,m_DEPRESSION_CrdPrv_low,m_DEPRESSION_CrdPrv_high,m_DIABETES_AgeAdjPrv_low,m_DIABETES_AgeAdjPrv_high,m_DIABETES_CrdPrv_low,m_DIABETES_CrdPrv_high,m_GHLTH_AgeAdjPrv_low,m_GHLTH_AgeAdjPrv_high,m_GHLTH_CrdPrv_low,m_GHLTH_CrdPrv_high,m_MHLTH_AgeAdjPrv_low,m_MHLTH_AgeAdjPrv_high,m_MHLTH_CrdPrv_low,m_MHLTH_CrdPrv_high,m_OBESITY_AgeAdjPrv_low,m_OBESITY_AgeAdjPrv_high,m_OBESITY_CrdPrv_low,m_OBESITY_CrdPrv_high
0,AK,Alaska,Aleutians East,3398,16.9,16.5,60.7,60.7,18.6,19.3,13.1,13.0,12.6,13.1,19.1,19.2,12.9,12.7,31.3,32.1,13.8,20.3,13.4,19.6,54.6,66.8,54.5,66.8,15.6,22.3,16.1,23.1,10.6,15.9,10.5,15.8,10.8,14.6,11.3,15.1,16.6,22.0,16.8,22.2,11.2,14.9,11.0,14.7,24.5,38.7,25.2,39.5
1,AK,Alaska,Aleutians West,5059,13.7,13.4,61.6,60.6,15.5,16.7,13.4,13.5,10.9,10.8,15.3,15.0,12.2,12.2,30.3,31.6,11.5,16.2,11.2,15.9,56.2,67.9,55.1,67.1,13.0,18.5,14.0,19.9,11.1,16.5,11.2,16.5,9.4,12.6,9.3,12.6,13.3,17.4,13.0,17.2,10.7,14.1,10.7,14.0,24.8,37.6,25.9,39.0
2,AK,Alaska,Anchorage,288121,10.4,10.3,62.9,62.7,14.9,15.0,19.5,19.7,7.8,7.9,13.5,13.5,14.5,14.6,32.9,33.0,8.5,12.7,8.5,12.6,58.9,66.6,58.7,66.4,12.3,17.7,12.4,17.7,17.1,22.1,17.3,22.4,6.8,8.9,6.9,9.0,11.7,15.6,11.7,15.5,12.8,16.3,12.9,16.4,29.0,37.0,29.1,37.1
3,AK,Alaska,Bethel,18557,19.2,19.1,62.6,61.2,35.4,35.5,21.2,21.9,15.9,14.8,30.0,29.0,21.4,22.4,43.1,42.8,15.2,23.4,15.0,23.5,57.1,67.6,55.7,66.4,29.9,40.7,29.7,40.7,17.7,25.1,18.3,26.0,13.8,18.1,12.7,16.8,26.1,34.2,25.0,33.1,18.6,24.2,19.4,25.3,36.3,49.7,35.9,49.4
4,AK,Alaska,Bristol Bay,838,10.3,9.8,61.4,64.2,18.7,18.8,18.8,18.1,8.6,10.7,14.4,15.9,15.4,14.4,35.6,36.4,8.4,12.4,8.1,11.8,55.4,66.9,58.4,69.5,15.3,22.1,15.5,22.3,15.5,22.6,14.9,21.8,7.3,10.1,9.1,12.6,12.4,16.6,13.7,18.3,13.4,17.6,12.5,16.5,28.5,42.9,29.3,43.9
5,AK,Alaska,Chugach,6941,9.8,9.3,61.1,63.3,15.4,15.7,20.1,19.7,7.3,8.7,12.8,13.7,14.6,13.8,31.4,32.2,7.8,12.2,7.5,11.4,58.1,64.2,60.2,66.2,12.8,18.1,13.0,18.4,18.1,22.3,17.7,21.9,6.5,8.1,7.7,9.6,11.1,14.5,11.8,15.7,13.1,16.1,12.4,15.2,28.1,34.7,28.8,35.7
6,AK,Alaska,Copper River,2630,11.8,11.1,61.4,64.9,19.4,18.9,20.0,19.0,8.6,10.8,15.8,17.4,16.0,14.6,34.9,35.3,9.6,14.4,9.1,13.4,57.6,64.8,61.3,68.2,16.4,22.6,16.0,22.1,17.8,22.4,16.9,21.3,7.6,9.6,9.6,12.1,13.9,18.2,15.1,19.9,14.3,17.7,13.1,16.2,30.7,39.4,31.0,39.6
7,AK,Alaska,Denali,1593,10.1,9.9,61.7,62.2,16.9,17.8,19.9,19.9,8.3,8.9,14.2,14.7,15.0,14.6,27.6,28.7,8.0,12.8,7.7,12.4,55.7,67.5,56.1,67.9,13.1,20.9,13.8,21.9,16.3,23.8,16.3,23.9,7.0,9.7,7.4,10.4,11.9,16.8,12.3,17.4,12.8,17.3,12.5,17.0,21.4,34.5,22.2,35.8
8,AK,Alaska,Dillingham,4772,15.4,15.3,59.0,58.8,29.1,29.1,20.3,20.6,13.3,13.5,23.8,23.8,19.0,19.3,40.7,40.6,12.3,19.1,12.0,18.8,53.3,64.5,53.1,64.4,24.0,34.1,24.0,34.1,16.6,24.1,16.8,24.6,11.3,15.2,11.4,15.4,20.2,27.4,20.1,27.3,16.3,21.6,16.6,22.0,33.7,47.9,33.5,47.7
9,AK,Alaska,Fairbanks North Star,95593,10.1,10.4,62.3,61.1,16.0,15.6,19.5,20.0,8.0,7.4,13.3,12.8,14.5,15.1,33.9,33.2,8.0,12.5,8.1,12.9,58.2,66.2,56.8,65.0,13.1,18.9,12.9,18.5,16.7,22.2,17.2,22.9,6.8,9.1,6.4,8.5,11.4,15.2,10.9,14.7,12.6,16.4,13.1,17.1,29.5,38.2,28.9,37.5


In [39]:
df_group.to_csv('BRFSS.csv', index=False)